In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx

import matplotlib.pyplot as plt

In [3]:
!python3 -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 6.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Users\T-GAMER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [7]:
model = 'en_core_web_sm'
NER = spacy.load(model)


In [101]:
book = [b for b in os.scandir('Book') if 'txt' in b.name]

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'Book'

In [96]:
# Check the first book
book_text= open('lightning_thief_the__percy_jac_-_rick_riordan.txt', encoding='ISO-8859-1').read()
# Set the limit
NER.max_length = 20000000
# Extract the Entity
book = NER(book_text)

In [33]:
displacy.render(book[108700:109000], style="ent", jupyter=True)

In [42]:
sentence= []

for i in book.sents:
    ent_list=(ent.text for ent in i.ents)
    sentence.append({"sentence":sent,'entity':ent_list})

In [43]:
sentence=pd.DataFrame(sentence)
sentence.head()

,sentence,entity
0,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C668078...
1,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C70C...
2,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C70C...
3,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C70C...
4,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C668078...


In [45]:
character_df=pd.read_excel('Character.xlsx')

In [46]:
character_df["first_name"] = character_df["Character"].apply( lambda x: x.split(" ",1)[0])

In [48]:
character_df.head()

,Character,first_name
0,Percy Jackson,Percy
1,Annabeth Chase,Annabeth
2,Grover Underwood,Grover
3,Thalia Grace,Thalia
4,Luke Castellan,Luke


In [53]:
# Filter out None Character rows from the data
def filter_out(entity_col, character_df):
    return [x for x in entity_col if x in list(character_df['first_name'])]

# Apply the function to
sentence['character_entity'] = sentence['entity'].apply(lambda x: filter_out(x, character_df))

In [54]:
sentence.head()

,sentence,entity,character_entity
0,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C668078...,[]
1,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C70C...,[]
2,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C70C...,[]
3,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C70C...,[]
4,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C668078...,[]


In [55]:
df_filter = sentence[sentence['character_entity'].apply(lambda x: len(x) > 0)].reset_index()

In [57]:
df_filter.head()

,index,sentence,entity,character_entity
0,73,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C19E...,[Grover]
1,85,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C192...,[Grover]
2,88,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C192...,[Grover]
3,103,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C1AA...,[Grover]
4,119,"(To, learn, more, about, him, ,, \n, visit, hi...",<generator object <genexpr> at 0x000001C65C1AA...,[Zeus]


In [62]:
# Window size (a number of senctences) to extract the relationship
window_size = 5

# Create an empty list to store the combined character_entities
char_list = []

# Iterate through rolling windows of the DataFrame
for window in df_filter['character_entity'].rolling(window=window_size):
    if window.size == window_size:
        char_list.extend(sum(window.dropna(), [])) 

In [64]:
# Keep only Unique list of character relationship
# Initialize a list to store unique elements
unique_list = []

# Initialize a variable to store the previous element
prev_element = None

# Iterate through char_list
for element in char_list:
    # Check if the current element is different from the previous one
    if element != prev_element:
        unique_list.append(element)
    # Update the previous element
    prev_element = element

In [65]:
unique_list[:10]

['Grover',
 'Zeus',
 'Grover',
 'Zeus',
 'Grover',
 'Zeus',
 'Grover',
 'Zeus',
 'Grover',
 'Zeus']

In [66]:
# Initialize 2 list
source = []
target = []

# Iterate through all elements of the list
for i in range(len(unique_list)-1):
    # Check if the current element is different from the previous one
    source.append(unique_list[i])
    target.append(unique_list[i+1])       

# Create a dataframe of this relationship direction
relationship_df = pd.DataFrame({"Source" : source,
                               "Target" : target})

In [67]:
relationship_df.head()

,Source,Target
0,Grover,Zeus
1,Zeus,Grover
2,Grover,Zeus
3,Zeus,Grover
4,Grover,Zeus


In [68]:
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df

,Source,Target
0,Grover,Zeus
1,Grover,Zeus
2,Grover,Zeus
3,Grover,Zeus
4,Grover,Zeus
...,...,...
4338,Annabeth,Tyson
4339,Annabeth,Tyson
4340,Annabeth,Tyson
4341,Annabeth,Tyson


In [69]:
# Set weight equal to 1 for each row
relationship_df["Weight"] = 1
# Aggregate data to create weight
relationship_df = relationship_df.groupby(["Source","Target"], sort=False, as_index=False).sum()

In [72]:
relationship_df=relationship_df.sort_values(by='Weight',ascending=False)


In [73]:
relationship_df

,Source,Target,Weight
8,Annabeth,Grover,653
1,Grover,Percy,209
4,Chiron,Percy,206
3,Chiron,Grover,196
11,Annabeth,Percy,192
...,...,...,...
50,Hades,Thalia,1
32,Clarisse,Grover,1
114,Demeter,Hades,1
80,Poseidon,Thalia,1


In [98]:
relationship_df.to_csv("Data/relationship_percy.csv", index=False) 

OSError: Cannot save file into a non-existent directory: 'Data'